In [1]:
training_data = [['Green', 3, 'Mango'],
                ['Yellow', 3, 'Mango'],
                ['Red', 1, 'Grape'],
                ['Red', 1, 'Grape'],
                ['Yellow', 3, 'Lemon']]

In [2]:
header = ["color", "diameter", "label"]

In [3]:
def unique_vals(rows, col):
    '''Find the unique values for a column in a dataset.'''
    return set([row[col] for row in rows])

In [4]:
unique_vals(training_data, 0)

{'Green', 'Red', 'Yellow'}

In [5]:
unique_vals(training_data, 2)

{'Grape', 'Lemon', 'Mango'}

In [6]:
def class_counts(rows):
    '''Counts the number of each type of example in a dataset.'''
    counts = {}    #a dictionary of label->count.
    for row in rows:
        #in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label]= 0
        counts[label] += 1
    return counts


In [7]:
class_counts(training_data)

{'Mango': 2, 'Grape': 2, 'Lemon': 1}

In [8]:
def is_numeric(value):
    '''Test if a value is numeric.'''
    return isinstance(value, int) or isinstance(value, float)

In [9]:
is_numeric(7)

True

In [10]:
is_numeric(0.4)

True

In [11]:
is_numeric("red")

False

In [32]:
class Question:
    '''A question is used to partition a dataset.
    This class just records a 'column number'(e.g., 0 for color) and a 'column value'(e.g., green). The 'match' method is used to compare the feature value in an example to the feature value stored in the question. See the demo beow.'''
    def _init(self, column, value):
        self.column = column
        self.value = value
    def match(self, example):
        #compare the feature value in an example to the 
        #feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
    def _repr_(self):
        #This is just a helper method to print
        #The question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" %(header[self.column], condition, str(self.value))
    def partition(rows, question):
        #Partitions a dataset. For each row in the dataset, check if it matches the question. If so, add it to 'true rows', otherwise, add it to 'false rows'.
        true_rows, false_rows = [], []
        for row in rows:
            if question.match(row):
                true_rows.append(row)
            else:
                false_rows.append(row)
        return true_rows, false_rows
        

In [33]:
true_rows, false_rows = partition(training_data, Question(0, 'Red'))


TypeError: Question() takes no arguments

In [17]:
false_rows

NameError: name 'false_rows' is not defined

In [18]:
def gini(rows):
    #calculate the gini impurity for a list of rows.
    
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl]/float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity


In [19]:
#lets see how gini impurity works.
#first, we'll look at a dataset with no mixing.
no_mixing = [['Mango'], ['Mango']]
gini(no_mixing)


0.0

In [20]:
lots_of_mixing = [['Mango'], ['Orange'], ['Grape'], ['Grapefruit'], ['Blueberry']]
gini(lots_of_mixing)

0.7999999999999998

In [21]:
def info_gain(left, right, current_uncertainity):
    '''Information Gain. The uncertainity of th starting node, minus the weighted impurity of two child nodes.'''
    p = float(len(left))/(len(left)+len(right))
    return current_uncertainity - p * gini(left) - (1-p) * gini(right)
    

In [24]:
#calculating th uncertainity of our training data.
current_uncertainity = gini(training_data)

In [25]:
#how much information do we gain by partitioning on 'Green'?
true_rows, false_rows = partition(training_data, Question(0, 'Green'))

NameError: name 'partition' is not defined

In [31]:
info_gain(true_rows, false_rows, current_uncertainity)


NameError: name 'true_rows' is not defined

In [34]:
#what about if we partition on red instead?
true_rows, false_rows = partition(training_data, Question(0, 'Red'))


TypeError: Question() takes no arguments

In [35]:
info_gain(true_rows, false_rows, current_uncertainity)

NameError: name 'true_rows' is not defined

In [36]:
def find_best_split(rows):
    '''Find the best question to ask by iteration over every feature/value and calculating the information gain.'''
    best_gain = 0 #keep track of the best information gain
    best_question = None #keep train of the feature/value that produced it.
    current_uncertainity = gini(rows)
    n_features = len(rows[0])-1 #number of columns
    for col in range(n_features): #for each feature
        values = set([row[col] for row in rows]) #unique values in the column
        for val in values: #for each value
            question = Question(col, val)
            #try splitting the dataset
            true_rows, false_rows = partition(rows, question)
            #skip this split if it doesn't divide the dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            #calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainity)
            #you can actually use > instead of >= here but i wanted the tree to look a certain way for our toy dataset.
            if gain>=best_gain:
                best_gain, best_question = gain, question
    return best_gain, best_question

                
            

In [37]:
#find the best question to ask first for our toy dataset.
best_gain, best_question = find_best_split(training_data)

TypeError: Question() takes no arguments

In [38]:
class leaf:
    '''A leaf node classifies data. This holds a dictionary of class(e.g., "Mango")-> no. of times it appears in the rows from the training data that reach this leaf.'''
    def _init_(self, rows):
        self.predictions = class_counts(rows)
class Decision_Node:
    '''A decision node asks a question. This holds a reference to the question and to the two child nodes.'''
    def _init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
    def build_tree(rows):
        '''Builds the tree. Try partitioning the dataset on each of the unique attribute, calculate the information gain, and return the question that produces the highest gain.'''
        gain, question = find_best_split(rows)
        
        #base case: no further info gain
        #since we can ask no further questions, we'll return a leaf.
        if gain==0:
            return Leaf(rows)
        
        #if we reach here we have found a useful feature/value to partition on.
        true_rows, false_rows = partition (rows, question)
        #recursively build the true branch
        true_branch = build_tree(true_rows)
        #recursively build the false branch
        false_branch = build_tree(false_rows)
        
        #return a question node. This records the best feature/value to ask at this point, as well as the branches to follow depending on the answer.
        return Decision_Node(question, true_branch, false_branch)
    def print_tree(node, spacing=""):
        '''World's most elegant tree printing function.'''
        #Base case: we've reached a leaf
        if isinstance(node, Leaf):
            print(spacing + "Predict", node.predictions)
            return
        #print the question at this node
        print(spacing + str(node.question))
        #call this function recursively on the true branch
        print(spacing + '--> True:')
        print_tree(node.true_branch, spacing + "  ")
        #call this function recursively on the false branch
        print(spacing+ '--> False:')
        print_tree(node.false_branch, spacing + "  ")
        
    def classify(row, node):
        #Base case: we've reached a leaf
        if isinstance(node, leaf):
            return node.predictions
        #decide whether to follow the true-branch or the false-branch. Compare the feature / value stored in the node, to the example we're considering.
        if node.question.match(row):
            return classify(row, node.true_branch)
        else:
            return classify(row, node.false_branch)
        
        

In [39]:
print_leaf(classify(training_data[0], my_tree))

NameError: name 'print_leaf' is not defined

In [40]:
print_leaf(classify(training_data[1], my_tree))

NameError: name 'print_leaf' is not defined

In [ ]:
if __name__ == '__main__':
    my_tree = build_tree(training_data)
    print_tree(my_tree)
    #evaluate
    testing_data = [['Green', 3, 'Mango'],
                ['Yellow', 4, 'Mango'],
                ['Red', 2, 'Grape'],
                ['Red', 1, 'Grape'],
                ['Yellow', 3, 'Lemon']]
    for row in testing_data:
        print("Actual: %s. predicted:%s" % (row[-1], print_leaf(classify(row, my_tree))))
#next steps
#add support for missing(or unseen) attributes. Prune the tree to prevent overfitting and add support for regression

        
    